# From twitter

Obtener tweets relacionados con el [sismo](http://ciencia.unam.mx/leer/652/-que-ocurrio-el-19-de-septiembre-de-2017-en-mexico-) vivido en la Ciudad de México en la fecha de $2029/\text{Sept}/19$.

In [1]:
import searchtweets
import pandas as pd
import numpy as np

Verificar que la versión del paquete [`searchtweets`](https://pypi.org/project/searchtweets-v2/) es **1.0.7**. La manera que se recomienda instalar el paquete es la siguiente:
+ `pip install searchtweets-v2`

In [2]:
searchtweets.__version__

'1.0.7'

In [3]:
from searchtweets import ResultStream, load_credentials, gen_request_parameters, collect_results

Verificar que se tiene el archivo `conf/local/credentials.yaml` con las credenciales necesarias para realizar el *scrapping* correspondiente a este notebook. En caso de que no se tengan se recomienda a acudir con los autores del proyecto.

In [4]:
search_args = load_credentials(filename="../conf/local/credentials.yaml",
                               yaml_key="search_tweets_v2",
                               env_overwrite=False)

In [5]:
query = gen_request_parameters("sismo cdmx", 
                               start_time = "2017-09-11",
                               end_time = "2017-12-31",
                               tweet_fields="id,created_at,conversation_id,in_reply_to_user_id,referenced_tweets,entities,text",
                               user_fields="id,username",
                               expansions="author_id",
                               results_per_call=10)

La siguiente línea es la versión antigua de generar el **scrapping**

In [6]:
#tweets = collect_results(query, max_tweets=50, result_stream_args=search_args) # change this if you need to

La siguiente línea es la versión actual de generar el **scrapping**:
+ `max_tweets`: indica la cantidad de tweets a generar

In [7]:
rs = ResultStream(request_parameters=query,
                  max_results=10,
                  max_pages=1,
                  max_tweets=50000,
                  **search_args)
print(rs)

ResultStream: 
	{
    "endpoint": "https://api.twitter.com/2/tweets/search/all",
    "request_parameters": {
        "query": "sismo cdmx",
        "max_results": 10,
        "start_time": "2017-09-11T00:00:00Z",
        "end_time": "2017-12-31T00:00:00Z",
        "tweet.fields": "id,created_at,conversation_id,in_reply_to_user_id,referenced_tweets,entities,text",
        "user.fields": "id,username",
        "expansions": "author_id"
    },
    "max_tweets": 50000
}


In [ ]:
results = list(rs.stream())

 HTTP Error code: 429: Rate limit exceeded
 | Too Many Requests
 Request payload: {'query': 'sismo cdmx', 'max_results': 10, 'start_time': '2017-09-11T00:00:00Z', 'end_time': '2017-12-31T00:00:00Z', 'tweet.fields': 'id,created_at,conversation_id,in_reply_to_user_id,referenced_tweets,entities,text', 'user.fields': 'id,username', 'expansions': 'author_id', 'next_token': '1jzu9lk96gu5npw44kgwrl2bhw4y896wrlg57exvs0vx'}
Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP Error code: 429: Rate limit exceeded
 | Too Many Requests
 Request payload: {'query': 'sismo cdmx', 'max_results': 10, 'start_time': '2017-09-11T00:00:00Z', 'end_time': '2017-12-31T00:00:00Z', 'tweet.fields': 'id,created_at,conversation_id,in_reply_to_user_id,referenced_tweets,entities,text', 'user.fields': 'id,username', 'expansions': 'author_id', 'next_token': '1jzu9lk96gu5npw44kgwrl2bhw4y896wrlg57exvs0vx'}
Rate limit hit... Will retry...
Will retry in 16 seconds...
 HTTP Error code: 429: Rate limit exceeded


 Request payload: {'query': 'sismo cdmx', 'max_results': 10, 'start_time': '2017-09-11T00:00:00Z', 'end_time': '2017-12-31T00:00:00Z', 'tweet.fields': 'id,created_at,conversation_id,in_reply_to_user_id,referenced_tweets,entities,text', 'user.fields': 'id,username', 'expansions': 'author_id', 'next_token': '1jzu9lk96gu5npw44kg2km113xvbzb7p97eemlvet3zx'}
Rate limit hit... Will retry...
Will retry in 64 seconds...
 HTTP Error code: 429: Rate limit exceeded
 | Too Many Requests
 Request payload: {'query': 'sismo cdmx', 'max_results': 10, 'start_time': '2017-09-11T00:00:00Z', 'end_time': '2017-12-31T00:00:00Z', 'tweet.fields': 'id,created_at,conversation_id,in_reply_to_user_id,referenced_tweets,entities,text', 'user.fields': 'id,username', 'expansions': 'author_id', 'next_token': '1jzu9lk96gu5npw44kg2km113xvbzb7p97eemlvet3zx'}
Rate limit hit... Will retry...
Will retry in 100 seconds...
 HTTP Error code: 429: Rate limit exceeded
 | Too Many Requests
 Request payload: {'query': 'sismo cdmx',

In [ ]:
res = pd.DataFrame(results)

In [ ]:
res.head(3)

In [ ]:
res.shape

In [ ]:
res.data[0]

In [ ]:
df_1 = pd.DataFrame()
for api in res.data:
    for tweet in api:
        if 'in_reply_to_user_id' not in tweet:
            tweet['in_reply_to_user_id'] = 'propio'
        if 'referenced_tweets' not in tweet:
            tweet['referenced_tweets'] = 'propio_original'
        if 'entities' not in tweet:
            tweet['entities'] = 'propio_original'
        lista = np.array([tweet['created_at'], tweet['id'], tweet['author_id'], 
                          tweet['conversation_id'], tweet['in_reply_to_user_id'], tweet['referenced_tweets'],
                          tweet['entities'],tweet['text']])
        temp = pd.DataFrame([lista], columns=['created_at', 'id', 'author_id', 'conversation_id', 'referenced_tweets',
                                              'reply_to_user_id', 'entities' ,'text'])
        df_1 = pd.concat([df_1,temp], axis=0)

Información del **tweet**

In [ ]:
df_1.head(6)

Información del usuario (**user**)

In [ ]:
df_2 = pd.DataFrame()
for api in res.includes:
    for tweet in api['users']:    
        lista = np.array([tweet['id'], tweet['username']])
        temp = temp = pd.DataFrame([lista], columns=['author_id', 'username'])
        df_2 = pd.concat([df_2,temp], axis=0)

In [ ]:
df_2['id_user'] = df_2.author_id.astype('str') + df_2.username

In [ ]:
df_2.head(6)

In [ ]:
df_2=df_2.drop_duplicates('id_user')

In [ ]:
df_2.shape

In [ ]:
df_2_clean = df_2[['author_id','username']]

In [ ]:
df = df_1.merge(df_2_clean, on='author_id', how='left')

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
#df.to_csv("sismo_cdmx_506.csv")